In [2]:
#!pip install selenium
#!pip install webdriver_manager

In [8]:
import numpy as np
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [86]:
options = Options()
chrome_driver_path = r'C:\Program Files\WebDriver\chromedriver-win64\chromedriver.exe'
# according download: https://googlechromelabs.github.io/chrome-for-testing/#stable | caution: chromeversion and chromewbdriver version have to match, to check chromeversion --> paste chrome://version/ into chrome 
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)

all_prices = []
all_car_names = []
all_infos = [] 

def slow_scroll(driver, scroll_pause_time=0.1, total_scroll_time=5):
    # Get the current scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    start_time = time.time()
    while True:
        # Calculate the current time elapsed
        current_time = time.time()
        if current_time - start_time >= total_scroll_time:
            break  # Stop scrolling after 5 seconds

        # Scroll down by a small amount
        driver.execute_script("window.scrollBy(0, 300);")  # Adjust the number of pixels per scroll if needed
        
        # Wait a short amount of time
        time.sleep(scroll_pause_time)

for page in range(0, 2):  # Loop, indicate between page 0 and 7885
    url = f'https://www.autoscout24.ch/de/s?pagination%5Bpage%5D={page}'
    driver.get(url)
    
    if page == 0:
        cookie_accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))
        )
        cookie_accept_button.click()
       
    # Slowly scroll through the website over 5 seconds
    slow_scroll(driver, 0.1, 3)

    price_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'p.chakra-text.css-bwl0or'))
    )
    prices = [element.text for element in price_elements]
    all_prices.extend(prices)

    name_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h2.chakra-heading.css-svmyrf'))
    )
    car_names = [element.text for element in name_elements]
    all_car_names.extend(car_names)

    info_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.css-1kcev50 p.chakra-text.css-rlwdn6'))
    )
    infos = [element.text for element in info_elements]
    all_infos.extend(infos)

    dates = all_infos[0::6]
    km = all_infos[1::6]
    transmission = all_infos[2::6]
    fuel = all_infos[3::6]
    ps = all_infos[4::6]
    consumption = all_infos[5::6]

    df = pd.DataFrame({
        'Car': all_car_names, 
        'Price': all_prices, 
        'Dates': dates, 
        'KM': km, 
        'Transmission': transmission, 
        'Fuel': fuel, 
        'PS': ps, 
        'Consumption': consumption})

driver.quit()

In [87]:
df

,Car,Price,Dates,KM,Transmission,Fuel,PS,Consumption
0,BMW 320d xDrive 48V Touring,CHF 51'900.–,06.2023,17'100 km,Automat,Mild-Hybrid Diesel/Elektro,190 PS (140 kW),5.4 l/100 km
1,BMW 120d xDrive Sport Line,CHF 36'900.–,05.2023,28'100 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km
2,SEAT Leon ST 2.0 TSI Cupra 290 DSG,CHF 21'900.–,03.2016,86'000 km,Halbautomatisches Getriebe,Benzin,290 PS (213 kW),6.6 l/100 km
3,AUDI A3 Sportback 35 TDI,CHF 36'900.–,06.2023,14'167 km,Halbautomatisches Getriebe,Diesel,150 PS (110 kW),-
4,MERCEDES-BENZ B 250 e AMG Line 8G-DCT,CHF 37'900.–,11.2020,9'500 km,Halbautomatisches Getriebe,Plug-in hybrid Benzin/Elektro,217 PS (160 kW),1.5 l/100 km
5,FORD S-Max 2.0 TDCi 180 Titanium FPS,CHF 19'700.–,03.2018,110'700 km,Halbautomatisches Getriebe,Diesel,180 PS (132 kW),5.2 l/100 km
6,VW Golf VIII 2.0 TSI GTI DSG,CHF 29'800.–,02.2021,23'900 km,Halbautomatisches Getriebe,Benzin,245 PS (180 kW),7 l/100 km
7,BMW 520d xDrive Touring Sport Line Steptronic,CHF 22'800.–,01.2018,129'009 km,Automat,Diesel,190 PS (140 kW),5.2 l/100 km
8,VW T-ROC 2.0 TDI 150PS DSG R-LINE BLACK STYLE,CHF 32'900.–,05.2023,12'900 km,Halbautomatisches Getriebe,Diesel,149 PS (110 kW),-
9,SEAT Ibiza 1.0 EcoTSI Move FR DSG,CHF 29'800.–,12.2023,2'500 km,Automatikgetriebe,Benzin,110 PS (81 kW),-
